<a href="https://colab.research.google.com/github/engineeringjoy/SynAnalyzer/blob/main/SynAnalyzer_ConvertImarisStatsFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SynAnalyzer_1.0.ConvertImarisStatsFile_Local.ipynb
Created by: JFranco | Created On: 5 AUG 2024 | Last Env: SynCounting | Last Run Date: 15 AUG 2024

This Python notebook is first module in the Synapse Analyzer pipeline. The purpose of the notebook is to take Excel files that were generated by Imaris, containing the statistics of all surfaces, and convert them into a simplified format that can be used by SynAnalyzer.ijm. The default values that are extracted are:

1. Surface ID as assigned by Imaris  
2. XYZ Positions
3. Volume for each surface

To change the voxel dimension settings, open a raw image in ImageJ and then go to the Image menu and select "Show Info." This information helps to ensure that thumbnails are generated at precisely the correct position to avoid ImageJ macro rounding errors.  

This code requires that the .xls files are in the original format output by Imaris. If "Spots" were used in Imaris, rather than surfaces, the code relating to volume extraction will need to adjusted.

Version Notes: This version has minor changes so that it runs locally in a SynAnalysis batch folder. 

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import xlrd

In [4]:
#                *** WHAT TO ANALYZE // WHERE TO GET/STORE **
# Key identifiers
batchID = 'Ntng1NIHL_2023'

# Voxel dimensions for converting XYZs (in um)
voxelWidth = 0.0248
voxelHeight = 0.0248
voxelDepth = 0.31

# Threshold for volume of surfaces to include. Must be greater than zero!
#   Volumes that are too small do not have a mean intensity and it will result in a runtime error
volThresh = 0.1

# Directories 
#   existing ones 
dirMain = '/Users/joyfranco/Dropbox (Partners HealthCare)/JF_Shared/Data/FromCollaborators/Copeland2024/'
dirBA = dirMain+batchID+'/'
dirData = dirBA+'ImarisStatsFiles/'

#   ones that need to be made    
dirSV = dirBA+'XYZCSVs/'        

In [5]:
#           *** INITIALIZE RUN SPECIFIC DIRECTORY ETC FOR STORING RESULTS **
# Create directory for storing spreadsheetS and summary plotS for this run
if not os.path.exists(dirSV): os.mkdir(dirSV)

In [6]:
#           *** GENERATE A LIST OF FILES THAT WILL BE CONVERTED **
# This list is based off the available Excel results. 
# Generate a list of all xls files added by the user
os.chdir(dirData)
files = glob.glob('*Syn.xls')
print(files)

['WSS_022.H.T1.02.Zs.4C.XYZ.PreSyn.xls', 'WSS_022.B.T2.02.ZS.4C.XYZ.PreSyn.xls', 'WSS_022.J.T1.01.Zs.4C.XYZ.PreSyn.xls', 'WSS_022.H.T1.02.Zs.4C.XYZ.PostSyn.xls', 'WSS_022.J.T1.01.Zs.4C.XYZ.PostSyn.xls', 'WSS_022.E.T1.01.Zs.4C.XYZ.PostSyn.xls', 'WSS_022.G.T3.02.Zs.4C.XYZ.PreSyn.xls', 'WSS_022.E.T1.01.Zs.4C.XYZ.PreSyn.xls', 'WSS_022.F.T2.02.Zs.4C.XYZ.PostSyn.xls', 'WSS_022.G.T3.02.Zs.4C.XYZ.PostSyn.xls', 'WSS_022.F.T2.02.Zs.4C.XYZ.PreSyn.xls', 'WSS_022.B.T2.02.ZS.4C.XYZ.PostSyn.xls', 'WSS_022.C.T3.02.ZS.4C.XYZ.PostSyn.xls', 'WSS_022.C.T3.02.ZS.4C.XYZ.PreSyn.xls']


In [7]:
# Iterate through the xls files, extract the relavent sheet, and reformat
for file in files:
    # Read in the sheets that correspond to each desired df
    dfXYZ = pd.read_excel(file,skiprows=1, sheet_name='Position')
    dfVol = pd.read_excel(file,skiprows=1, sheet_name='Volume')
    dfIMO = pd.read_excel(file, sheet_name='Intensity Mean Ch=1 Img=1')
    dfIMT = pd.read_excel(file,sheet_name='Intensity Mean Ch=2 Img=1')
    dfIMTh = pd.read_excel(file,sheet_name='Intensity Mean Ch=3 Img=1')
    dfIMF = pd.read_excel(file,sheet_name='Intensity Mean Ch=4 Img=1')
    
    # Reformat each df to make it a proper df rather than sheet
    dfXYZ = dfXYZ.drop(columns=['Unit', 'Category', 'Collection', 'Time'])
    dfXYZ.set_index('ID', inplace=True)
    dfVol = dfVol.drop(columns=['Unit', 'Category','Time'])
    dfVol.set_index('ID', inplace=True)
    
    #dfIMO = dfIMO.drop(columns=['Unit', 'Category', 'Channel','Image','Time'])
    dfIMO.columns = dfIMO.iloc[0]
    dfIMO.drop(dfIMO.head(1).index, inplace=True)
    dfIMO.set_index('ID', inplace=True)
    
    #dfIMT = dfIMT.drop(columns=['Unit', 'Category', 'Channel','Image','Time'])
    dfIMT.columns = dfIMT.iloc[0]
    dfIMT.drop(dfIMT.head(1).index, inplace=True)
    dfIMT.set_index('ID', inplace=True)
    
    #dfIMTh = dfIMTh.drop(columns=['Unit', 'Category', 'Channel','Image','Time'])
    dfIMTh.columns = dfIMTh.iloc[0]
    dfIMTh.drop(dfIMTh.head(1).index, inplace=True)
    dfIMTh.set_index('ID', inplace=True)
    
    #dfIMF = dfIMF.drop(columns=['Unit', 'Category', 'Channel','Image','Time'])
    dfIMF.columns = dfIMF.iloc[0]
    dfIMF.drop(dfIMF.head(1).index, inplace=True)
    dfIMF.set_index('ID', inplace=True)
    
    # Need to transfer the information about the volume over the XYZ df without
    #. assumptions about df order
    for id in list(dfXYZ.index.values):
        # Get the volume associated with this ID from the volume df
        vol = dfVol['Volume'][id]
         

        # Filter out any surfaces with volume below threshold
        if (vol > volThresh):
            dfXYZ.at[id, 'Volume_um3'] = vol
            
            # First convert position from XYZ in microns to XYZ in voxels
            xPos = dfXYZ['Position X'][id]
            yPos = dfXYZ['Position Y'][id]
            zPos = dfXYZ['Position Z'][id]
    
            dfXYZ.at[id, 'Position X (voxels)'] = xPos/voxelWidth
            dfXYZ.at[id, 'Position Y (voxels)'] = yPos/voxelHeight
            dfXYZ.at[id, 'Position Z (voxels)'] = round(zPos/voxelDepth)
            
            
    
            # Get the ch1 mean intensity
            dfXYZ.at[id, 'uIntCh_1'] = dfIMO['Intensity Mean'][id]
            # Get the ch2 mean intensity
            dfXYZ.at[id, 'uIntCh_2'] = dfIMT['Intensity Mean'][id]
            # Get the ch3 mean intensity
            dfXYZ.at[id, 'uIntCh_3'] = dfIMTh['Intensity Mean'][id]
            # Get the ch4 mean intensity
            dfXYZ.at[id, 'uIntCh_4'] = dfIMF['Intensity Mean'][id]
        else:
            dfXYZ.drop(id, inplace = True)

    # Save the dataframe as a csv file    
    dfXYZ.to_csv(dirSV+file.split('.x')[0]+'.csv')